In [1]:
#Requires Python 3.8+

# Imports

In [2]:
import random
import secrets
from decimal import *

# Functions

Returns the GCD of 2 numbers

In [3]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Continued Fractions

In [4]:
from fractions import Fraction

def continued_fractions(convergence):
    
    convergence_list = []
    for i in range(1, len(convergence) + 1):
        convergence_list.append(convergence[:i])
    
    fractions = []
    
    for c in convergence_list:
        fractions.append(recursive_continued_fraction(c.copy()))
    
    return fractions
    
def recursive_continued_fraction(convergence):
    
    a = convergence.pop(0)
    
    if len(convergence) == 0:
        return a
    else:
        a = a + Fraction(1, (recursive_continued_fraction(convergence)))
        return a
    return a

In [5]:
def continued_fractions_expansion(a, b):
    expansion = []
    while(b):
        expansion.append(a // b)
        a, b = b, a % b 
    return expansion

Rabin-Miller Primality Test<br>
Checks if a number is prime, p is the number to be tested, s is the number of rounds<br>
Number of rounds should ensure that the probability of p not being prime is < 2^(-80) 

In [6]:
def RMPT(p, s):
    q = p - 1
    k = 0
    while q % 2 != 1:
        q //= 2
        k += 1
        
    for _ in range(s):
        a = random.randrange(3, p - 2)      
        while gcd(a, p) != 1:
            a = random.randrange(3, p - 2)
        
        z = pow(a, q, p)
        
        if z == 1 or z == p - 1:
            continue #Not a witness, pick another a
        
        for i in range(1, k):
            z = pow(a, pow(2, i)*q, p)
            if z == p - 1:
                break #Not a witness, pick another a
        else:
            return False  #a^((2^K)*Q)modP != -1 for all values of K, it is a witness, therefore its definitely not prime
        
    return True #After S iterations still not a composite, chance to be a prime

Converts a string to ascii

In [7]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

Converts ascii back to string

In [8]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Generates the decryption key using public key e and 2 prime numbers p and q

In [9]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

Encrypts a number with public key e

In [10]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Encrypts a number using private key d

In [11]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

Continuously generates random bit_length numbers<br>
If number is even, adds one to make it odd<br>
Tries to factorize the number with small primes to determine if composite<br>
Uses Rabin-Miller Primality Test if it is still possible to be a prime number

In [12]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

# RSA 1024

Generate 2 random 1024 bit primes

In [13]:
p = Generate_Prime(1024, 3)

44766954677586077999037341459065199180495034222178160616135997111460866687785624936836160949057483998284545472916477804841128189268153096340621271193131742751749173153959857256123356218867595064090864328164930868675696585703532402313754860839998014353526607525112438299000673505758441223598058535431008536127


In [14]:
q = Generate_Prime(1024, 3)

85918907526637935690325256161579873701713503565353628239700380362994063548575393648675715822214794776304258132868091267717089760489050679167173264725935058989704584767063096210146358297999113855982071843616838516542295320273897154794453716346017830980147527063602013145124816313902633182395653927821397283259


In [15]:
N = p * q

So lets say you are designing a program that has a high emphasis on performance, in particular having high performance decryption. Using a large decryption exponent would mean it would take longer.

Since e and d are interchangable for the purposes of encrypting and decrypting (Provided you have not published the public exponent yet) 

c = m^d mod N
m = c^e mod N

c = m^e mod N
m = c^d mod N

Lets say that instead of using e to encrypt your message, you instead choose to use a much larger d to encrypt, using the smaller e to decrypt the message.

This should increase the performance!

But doing so would allow someone to use Wiener's attack to retrieve your decryption exponent since the decryption exponent is smaller than 1/3 * N ^ 1/4

# Wiener's attack

First lets randomly generate a small decryption exponent

In [54]:
phi_N = (p-1) * (q-1)

while(True):
    d = random.getrandbits(32)
    if gcd(d, phi_N) == 1:
        break;

In [55]:
print(d)

4085401199


Compute the encryption exponent

In [56]:
e = RSA_generate_D(d, p, q)

In [58]:
print(e)

3171931533469591756780326733493661188545004634935736429330942597525923233614015324715593262853307417667488754617574628271823291190919732933545738578649029411186676390454687835993641828846133347997461937894176210302899765003878221514809470554104678200626230959097554055656763118248254634185739459038513710185251929267730539995014226790459916801830983118333418374698686589788723309280258162867985899835416379122600327112947547635917372346036082165093322789064841153887783370243306774842227290419921432679262753665706289747139406224341614486947980982298594399177615125744514403398844240762818137435770377544880422249279


In [59]:
message = "Why using small decryption exponent is bad"

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

87104121032117115105110103032115109097108108032100101099114121112116105111110032101120112111110101110116032105115032098097100


416

Encrypt message using e

In [60]:
encrypted_message = RSA_encrypt(ascii_message, e, N)

In [61]:
print(encrypted_message)

1652639568623495401814885327801839496969237098295600054285406835380869843366557636126270115920896221747173299315973818203545880579013690270948985648418319176057804783084628117457459215841090390878389877071715187177897880507256474124857524467404164826669799656546764163258497935574617277500572983095415269836527976978332495693541293589368542088111666563003460902976591027975479981491346279725030200898825679543727413749348642737687552132802845566092500055380148485402157605056330784135301002801297220772950814640544122142953876530012440773867880058034871443056161251863124259508205019023184742840833592119376377311495


In [62]:
print(convert_to_string(encrypted_message))

ʲĎδϙʈƢĿ°9̤̏Tɴuǉǋ×͉ZƆͮƅͭGˋ»±΁ͰǻĀǚ|͙ȌǓƔ¤̺ʝ̟ʐȢ˼£ĂǱΧȾɩĕǴȼϗ_Ɵč̈́ȏϐϒŌǯʵȝĥɍŰȞXoʚȳǌΆϐɏϏǟϕǫŚė˕È΂̹ʧȟ˗Ɲ˭ŜʂˡʯȨ̢͍ȶ\Ǵ7żǥƒɝ8Ŋ̐ĭ̡ĩǕζ̮ʀȠzιͬȒƸ̅ͣͰ:"ͧƻ8¡û͟|ăǼÍ¸˦͈́ɐwŸŹķǯ


I can now calculate the convergents of e/N

In [63]:
expansion = continued_fractions_expansion(e, N)
fractions = continued_fractions(expansion)

One of the denominators will be d

In [65]:
denominators = []
for i in fractions:
    denominators.append(i.denominator)

(m^e)^d mod N

= m^1 mod N

= m mod N

We can encrypt a random message with e and test out all the denominators to see which one will result in getting the same message back

In [66]:
random_message = pow(17, e, N)

In [67]:
for denominator in denominators:
    if pow(random_message, denominator, N) == 17 :
        print(denominator)
        break;

4085401199


We found d!